In [168]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/utkmls-twitter-spam-detection-competition/sample_submission.csv
/kaggle/input/utkmls-twitter-spam-detection-competition/train.csv
/kaggle/input/utkmls-twitter-spam-detection-competition/test.csv


In [169]:
train = pd.read_csv('../input/utkmls-twitter-spam-detection-competition/train.csv')
test = pd.read_csv('../input/utkmls-twitter-spam-detection-competition/test.csv')
sample = pd.read_csv('/kaggle/input/utkmls-twitter-spam-detection-competition/sample_submission.csv')

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14899 entries, 0 to 14898
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Tweet       14899 non-null  object 
 1   following   14741 non-null  float64
 2   followers   14882 non-null  float64
 3   actions     11462 non-null  float64
 4   is_retweet  14898 non-null  float64
 5   location    12888 non-null  object 
 6   Type        14899 non-null  object 
 7   Unnamed: 7  2 non-null      object 
dtypes: float64(4), object(4)
memory usage: 931.3+ KB


개인적으로 GPU를 사용할 환경을 갖추지 않았기 때문에, Deep learning을 사용하는 것보다, sklearn이 제공하는 ML model들을 사용하여 해당 task를 해결하려 했습니다. Inference로 주신 notebook에서 충분히 모델 학습을 위한 코드가 제공되어 있기 때문에 이렇게 시도해보고 싶었습니다.

In [170]:
# But it's better to use pretrained word vector than not.
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [171]:
# Prepare ML models for classification.
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn import metrics

In [172]:
# Check the label
train['Type'].describe()

count       14899
unique          3
top       Quality
freq         7454
Name: Type, dtype: object

In [173]:
# This is spam classification problem but has 'South Dakota' which is region name
train['Type'].unique()

array(['Quality', 'Spam', 'South Dakota'], dtype=object)

In [174]:
# drop rows which has 'south dakota' label
train.drop(train[(train['Type'] == 'South Dakota')].index, inplace=True)
train['Type'].describe()

count       14897
unique          2
top       Quality
freq         7454
Name: Type, dtype: object

In [175]:
#think contain LINK will be spam
https_sent = train['Tweet'].str.contains('https://') #But some of them are """Quality"""
train[https_sent][:30].to_string()

"                                                                                                                                                          Tweet  following  followers  actions  is_retweet                       location     Type Unnamed: 7\n5                         #amms Samantha Bee Tries to ATTACK Trump, Instead Insults Millions of Americans https://t.co/YcGbUqjpzY #amis https://t.co/ngYzOjidr9       29.0        7.0    389.0         0.0                        England     Spam        NaN\n7                                                                                       Politicshttps://twitter.com/ngunjiriwambugu/status/732586106936565760\xa0…        0.0        0.0      0.0         0.0                            NaN  Quality        NaN\n8                                     Banking on Brexit: is it time to Invest in the UK? by Colin Ward (@ColinWard18) https://t.co/A34SVbwiIi #iartg #amwriting      530.0      849.0   8288.0         1.0                   Kuala Lump

In [176]:
https_tco_sent = train['Tweet'].str.contains('https://t.co|http://t.co')
train[https_tco_sent][:30].to_string()

"                                                                                                                                                          Tweet  following  followers  actions  is_retweet                        location  Type Unnamed: 7\n5                         #amms Samantha Bee Tries to ATTACK Trump, Instead Insults Millions of Americans https://t.co/YcGbUqjpzY #amis https://t.co/ngYzOjidr9       29.0        7.0    389.0         0.0                         England  Spam        NaN\n8                                     Banking on Brexit: is it time to Invest in the UK? by Colin Ward (@ColinWard18) https://t.co/A34SVbwiIi #iartg #amwriting      530.0      849.0   8288.0         1.0                    Kuala Lumpur  Spam        NaN\n9                                                                                         They are serving and protecting #5WordRegrets https://t.co/k6VlhS6G65     6601.0     8805.0  10712.0         0.0              Sheffield, England  Spam

In [177]:
#After details of Tweet, looks t.co LINK are almost spam
tco_quality = train[https_tco_sent]['Type'].str.contains('Quality')
train[https_tco_sent][tco_quality].to_string()

'                                                                                                                                            Tweet  following  followers  actions  is_retweet location     Type Unnamed: 7\n2158  RT@dailybigdata Big Data (Structured, Semi-Structured and Unstructured) Market 2016 - Global Forecasts to 2021 for t... https://t.co/9jGdgH        0.0        0.0      0.0         0.0  Jakarta  Quality        NaN'

In [178]:
#Looks its outlier, delete it.
train.drop(train[(train['Type'] == 'Quality') & (train['Tweet'].str.contains('https://t.co|http://t.co'))].index, inplace=True)
train

,Tweet,following,followers,actions,is_retweet,location,Type,Unnamed: 7
0,Good Morning Love @LeeBrown_V,0.0,0.0,0.0,0.0,"Pennsylvania, USA",Quality,NaN
1,'@realDonaldTrump @USNavy RIP TO HEROES',42096.0,61060.0,5001.0,0.0,"South Padre Island, Texas",Spam,NaN
2,Haven't been following the news but I understa...,0.0,0.0,NaN,0.0,Will never be broke ever again,Quality,NaN
3,pic.twitter.com/dy9q4ftLhZ What to do with pap...,0.0,0.0,0.0,0.0,Mundo,Quality,NaN
4,#DidYouKnow ► Mahatma Gandhi made a brief visi...,17800.0,35100.0,NaN,0.0,"Nottingham, England",Quality,NaN
...,...,...,...,...,...,...,...,...
14894,"#AllWentWrongWhen I told my hair stylist to ""g...",695.0,533.0,868.0,1.0,United States,Spam,NaN
14895,"They don't have to like you, and you don't hav...",0.0,0.0,0.0,0.0,NaN,Quality,NaN
14896,#Miami Graham Nash Live at Parker Playhouse #...,5647.0,15091.0,5823.0,0.0,United States,Spam,NaN
14897,@bethannhamilton is in the business of one-upp...,0.0,0.0,NaN,0.0,"Southgate, MI",Quality,NaN


In [179]:
https_tco_sent = train['Tweet'].str.contains('https://t.co|http://t.co')
spams = train[~https_tco_sent]['Type'] == 'Spam'
#To discover how to annotate 'spam'
train[~https_tco_sent][spams]

,Tweet,following,followers,actions,is_retweet,location,Type,Unnamed: 7
1,'@realDonaldTrump @USNavy RIP TO HEROES',42096.0,61060.0,5001.0,0.0,"South Padre Island, Texas",Spam,NaN
10,Beauty is not in the face,230.0,32.0,285.0,0.0,Germany or the Digital World,Spam,NaN
11,Dancing for your pleasure.,262.0,79.0,961.0,0.0,"Tokyo, Japan",Spam,NaN
13,#ICantBeUrFriendBecause it's better to be alon...,669.0,1128.0,1212.0,0.0,"40.255588,-76.877092",Spam,NaN
14,#SextingWentWrongWhen I tried to text Mcdonald...,4089.0,3416.0,7786.0,1.0,follow our Instagram,Spam,NaN
...,...,...,...,...,...,...,...,...
14882,Setting goals is the first step in turning the...,543.0,1012.0,6038.0,1.0,United States,Spam,NaN
14886,Ex-Mexican foreign minister Espinosa nominated...,8617.0,12301.0,20938.0,0.0,United States,Spam,NaN
14891,Loading back my moods #ChristmasAftermath,3445.0,3268.0,4546.0,1.0,United States,Spam,NaN
14894,"#AllWentWrongWhen I told my hair stylist to ""g...",695.0,533.0,868.0,1.0,United States,Spam,NaN


In [180]:
# Looks #news, #local are usually SPAM.
news_locals = train[~https_tco_sent][spams]['Tweet'].str.contains('#news|#local')
suspicious = train[~https_tco_sent][spams][news_locals]['Type'] != 'Spam'
train[~https_tco_sent][spams][news_locals][suspicious]

,Tweet,following,followers,actions,is_retweet,location,Type,Unnamed: 7


In [181]:
maybe_spam = train['Tweet'].str.contains('https://t.co|http://t.co|#news|#local')
suspicious = train[maybe_spam]['Type'] != 'Spam'
train[maybe_spam][suspicious]

,Tweet,following,followers,actions,is_retweet,location,Type,Unnamed: 7
5466,BRIEF-Elephant Talk posts Q1 loss of $0.03/sha...,NaN,0.0,0.0,0.0,Singapore,Quality,NaN
7702,Baudry among eight leaving Orient http://footy...,0.0,0.0,0.0,0.0,UK,Quality,NaN
13379,Strangers Rally To Help Mum 'Forced Onto Stree...,0.0,0.0,0.0,0.0,NaN,Quality,NaN


In [182]:
# Above rows are removed to train better.
train.drop(train[maybe_spam][suspicious].index, inplace=True)
train

,Tweet,following,followers,actions,is_retweet,location,Type,Unnamed: 7
0,Good Morning Love @LeeBrown_V,0.0,0.0,0.0,0.0,"Pennsylvania, USA",Quality,NaN
1,'@realDonaldTrump @USNavy RIP TO HEROES',42096.0,61060.0,5001.0,0.0,"South Padre Island, Texas",Spam,NaN
2,Haven't been following the news but I understa...,0.0,0.0,NaN,0.0,Will never be broke ever again,Quality,NaN
3,pic.twitter.com/dy9q4ftLhZ What to do with pap...,0.0,0.0,0.0,0.0,Mundo,Quality,NaN
4,#DidYouKnow ► Mahatma Gandhi made a brief visi...,17800.0,35100.0,NaN,0.0,"Nottingham, England",Quality,NaN
...,...,...,...,...,...,...,...,...
14894,"#AllWentWrongWhen I told my hair stylist to ""g...",695.0,533.0,868.0,1.0,United States,Spam,NaN
14895,"They don't have to like you, and you don't hav...",0.0,0.0,0.0,0.0,NaN,Quality,NaN
14896,#Miami Graham Nash Live at Parker Playhouse #...,5647.0,15091.0,5823.0,0.0,United States,Spam,NaN
14897,@bethannhamilton is in the business of one-upp...,0.0,0.0,NaN,0.0,"Southgate, MI",Quality,NaN


In [183]:
test

,Id,Tweet,following,followers,actions,is_retweet,location
0,0,"Cops shoot blacks almost everyday, and there a...",5509.0,5977.0,8062.0,1,ですpacito
1,1,"#HAPPYTAEYANGDAY Oppa,happy birthday to you, ...",0.0,0.0,NaN,0,NaN
2,2,RT @HoneyBadger10: Panthers in the super bowl....,471.0,229.0,747.0,0,"B'klyn, NYC, LA"
3,3,Sorry my Twitter keeps posting my retweets a m...,0.0,79800.0,48.0,0,NaN
4,4,Heart attack causes and symptoms are different...,902.0,5797.0,4270.0,0,Chicago
...,...,...,...,...,...,...,...
780,780,New Project For NLex Please @DreamscapePH @abs...,0.0,0.0,0.0,0,NaN
781,781,Actor DiCaprio joins growing movement to dives...,8753.0,12323.0,9023.0,0,United States
782,782,Just overheard 'looping in' 'reaching out' and...,45.0,49.0,60.0,1,United Kingdom
783,783,Photoshoot time! #music #band #photoshootpic.t...,0.0,0.0,0.0,0,"Southampton, England"


In [184]:
test_spam = test['Tweet'].str.contains('https://t.co|http://t.co|#news|#local')
test[test_spam]

,Id,Tweet,following,followers,actions,is_retweet,location
7,7,Our Latino Olympians: Puerto Rican Field Hocke...,2226.0,2737.0,1654.0,1,everywhere #teamCHOCOLATE
8,8,GBI: Man dies after being tazed by police duri...,8550.0,17694.0,7358.0,0,"Москва, Россия"
9,9,RT bocavista2016: DEAR #AMERICA #FakeNews say...,22.0,7.0,964.0,0,in way too many fandoms
13,13,#Oakland voices: Trip to South Africa for West...,6490.0,10242.0,7363.0,0,"Anfield, Liverpool"
17,17,'Little Giant' gets the spotlight thanks to Wa...,2423.0,1072.0,446.0,0,The here and now
...,...,...,...,...,...,...,...
769,769,Federal court blocks expansion of Obama's over...,6952.0,19608.0,40959.0,0,United States
771,771,According to Liberals buzzfeed is a legitimate...,42289.0,56601.0,4436.0,0,United States
772,772,Search For Suspect After Brentwood Pharmacy Ro...,2534.0,5803.0,5471.0,0,United States
774,774,Why don't you go find some tampon dispensers t...,2488.0,2065.0,5272.0,1,United States


In [185]:
train = train.drop(['following', 'followers','actions', 'is_retweet', 'location', 'Unnamed: 7'], axis=1)
train

,Tweet,Type
0,Good Morning Love @LeeBrown_V,Quality
1,'@realDonaldTrump @USNavy RIP TO HEROES',Spam
2,Haven't been following the news but I understa...,Quality
3,pic.twitter.com/dy9q4ftLhZ What to do with pap...,Quality
4,#DidYouKnow ► Mahatma Gandhi made a brief visi...,Quality
...,...,...
14894,"#AllWentWrongWhen I told my hair stylist to ""g...",Spam
14895,"They don't have to like you, and you don't hav...",Quality
14896,#Miami Graham Nash Live at Parker Playhouse #...,Spam
14897,@bethannhamilton is in the business of one-upp...,Quality


In [186]:
# max train tweet tokens = 274, so that using only 256 tokens
# because usually tweet is less than 100 tokens
# too much padding concerns model.

def encoding(sentences, max_length = 256):
    tokenized = []
    for sent in sentences:
        tok = tokenizer.encode(
            sent,
            max_length=max_length,
            truncation=True
        )
        padding_length = max_length - len(tok)
        tok += [0] * padding_length
        tokenized.append(tok)
    return tokenized

input_tokens = encoding(train['Tweet'])

labels = []
for label in train['Type']:
    if label == 'Spam':
        labels.append(1)
    else:
        labels.append(0)
print(len(input_tokens), len(labels))

14893 14893


In [187]:
# To choose which model is better to classifying it.
train_x, train_y = input_tokens[:12000], labels[:12000]
test_x, test_y = input_tokens[12000:], labels[12000:]

In [188]:
dt = DecisionTreeClassifier()
dt.fit(train_x, train_y)
dt_pred = dt.predict(test_x)
print("Decision Tree accuracy: {}".format(metrics.accuracy_score(test_y,dt_pred)))

Decision Tree accuracy: 0.694089180781196


In [189]:
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(train_x,train_y)
rfc_pred = rfc.predict(test_x)
print("Random forest accuracy: {}".format(metrics.accuracy_score(test_y,rfc_pred)))

Random forest accuracy: 0.8392671966816454


In [190]:
svc = SVC()
svc.fit(train_x,train_y)
svc_pred = rfc.predict(test_x)
print("Support Vector Machine: {}".format(metrics.accuracy_score(test_y,svc_pred)))

Support Vector Machine: 0.8392671966816454


In [191]:
cbc = CatBoostClassifier(silent=True, depth=6,iterations=300)
cbc.fit(train_x, train_y)
cbc_pred = cbc.predict(test_x)
print("Cat Boost accuracy: {}".format(metrics.accuracy_score(test_y,cbc_pred)))

Cat Boost accuracy: 0.8924991358451434


In [192]:
"""
Even this simple algorithm got over 80 accuracy score.
preds = []
for sent in test['Tweet']:
    if 'http://t.co' in sent:
        preds.append('Spam')
    elif 'https://t.co' in sent:
        preds.append('Spam')
    else:
        preds.append('Quality')
"""

"\nEven this simple algorithm got over 80 accuracy score.\npreds = []\nfor sent in test['Tweet']:\n    if 'http://t.co' in sent:\n        preds.append('Spam')\n    elif 'https://t.co' in sent:\n        preds.append('Spam')\n    else:\n        preds.append('Quality')\n"

In [193]:
ids = test['Id']
test_input_tokens = encoding(test['Tweet'])

#CBC is best.
CBC = CatBoostClassifier(silent=True, depth=6,iterations=300)
CBC.fit(input_tokens,labels)
result = CBC.predict(test_input_tokens)
CBC_preds = []
for r in result:
    if r == 0:
        CBC_preds.append('Quality')
    else:
        CBC_preds.append('Spam')

df = pd.DataFrame(CBC_preds, columns=['Type'])
result = pd.concat([ids,df], axis=1)
result.to_csv('submission.csv', index=False)
result

,Id,Type
0,0,Spam
1,1,Quality
2,2,Spam
3,3,Quality
4,4,Spam
...,...,...
780,780,Quality
781,781,Spam
782,782,Quality
783,783,Quality
